In [1]:
# go up a directory
import os
os.chdir('..')

In [2]:
# get cwd
os.getcwd()

In [1545]:
import glob
import torch
from openxai.dataloader import return_loaders, get_tokenizer_and_vocab
import numpy as np
from openxai.ML_Models.LR.model import LogisticRegression
import openxai.ML_Models.ANN.MLP as model_MLP
import openxai.ML_Models.ANN.Text_MLP as model_MLP_Text
import datetime
from sklearn.metrics import auc
from utils import get_model_names

In [1546]:
def calculateFaithfulnessAUC_text(test_sentences, explanations, text_classifier, min_idx, max_idx, max_k, do_pgu=False, do_random_baseline=False):
    PG_AUC = []
    # sentence_pointer = 0
    # explanation_pointer = 0
    # skipped = False
    for index, _ in enumerate(test_sentences):
        # if skipped:
        #     skipped = False
        #     sentence_pointer += 1
        #     continue
        test_sentence = test_sentences[index]
        # print('test_sentence', test_sentence)
        # # print('sentence_pointer', sentence_pointer)
        if index in bad_reply_indices + manual_skip_sentence_index:
#             print('-------------------\n\n\n')
#             print(f'Skipping {index} because it has a bad reply from the LLM, {all_exps[index]}')
            # sentence_pointer += 1
            # explanation_pointer += 1
            continue

#         # print('len(tokenizer(test_sentence))',len(tokenizer(test_sentence)))
        tokenized_sentence = tokenizer(test_sentence)
        num_tokens = len(tokenized_sentence)
#         print('num_tokens', num_tokens)
        if num_tokens < min_num_tokens_needed:
#             print(f'Skipping {index} because it has too few tokens ({num_tokens})')
#             print(f'Need at least {min_num_tokens_needed} tokens')
            # sentence_pointer += 1
            
            # skipped = True
            continue
        if index == max_idx:
            break
        if max_k > 1:
            auc_x = np.arange(max_k) / (max_k - 1)
        PG = []
        for top_k in range(1, max_k + 1):
#             print('explanation:', explanations[index][0][:top_k])
            PG.append(
                PG_words(test_sentence, explanations[index][0][:top_k], text_classifier, do_pgu, do_random_baseline)) 
        if max_k > 1:
            PG_AUC.append(auc(auc_x, PG))
        else:
            PG_AUC.append(PG)
        # explanation_pointer += 1
        # sentence_pointer += 1
    return PG_AUC

def PG_words(test_sentence, topk_exp_words, text_classifier, do_pgu, do_random_baseline=False):
#     print('test_sentence', test_sentence)
#     print('IN PG WORDS FUNCTION')
    tokenized_input = torch.tensor([voc[t] for t in tokenizer(test_sentence)])
    with torch.no_grad():
        pred_original = text_classifier(tokenized_input.unsqueeze(0))

    if do_random_baseline:
        # Create a mask where each element is True if it is NOT in values_to_remove
        mask = torch.rand(tokenized_input.shape) > 0.5
        if not do_pgu:
            while mask.sum() == 0:
                mask = torch.rand(tokenized_input.shape) > 0.5
        else:
            while mask.sum() == len(tokenized_input):
                mask = torch.rand(tokenized_input.shape) > 0.5

    else:
        # find the indices where the topk words are in the sentence
#         # print('topk_exp_words', topk_exp_words)
        voc_values_to_remove = []
#         print('topk_exp_words', topk_exp_words)
        for word in topk_exp_words:
#             print('word', word)
            top_k_idx_to_remove = voc[tokenizer(word)[0]]
            voc_values_to_remove.append(top_k_idx_to_remove)
        
        # Create a mask where each element is True if it is NOT in values_to_remove
#         print('tokenized_input', tokenized_input)
#         print('voc_values_to_remove', voc_values_to_remove)
        mask = ~torch.isin(tokenized_input, torch.tensor(voc_values_to_remove))
    
    if do_pgu:
#         print('flipping mask!')
#         print('mask', mask)
        # flip the mask
        mask = ~mask
#         print('flipped mask', mask)
        
    # Apply the mask to get the filtered tensor
    filtered_tokenized_input = tokenized_input[mask]
#     print('tokenized_input', tokenized_input)
#     print('mask', mask)
#     print('filtered_tokenized_input', filtered_tokenized_input)
    
    with torch.no_grad():
        pred_removed = text_classifier(filtered_tokenized_input.unsqueeze(0))
        
#     print('pred_original', pred_original)
#     print('pred_removed', pred_removed)
    PG = torch.abs(pred_original.squeeze() - pred_removed.squeeze())[0] # [0] - take first class. assume binary classifier (this is what openxai does) 
#     print("FINISHED PG WORDS FUNCTION")
    return PG

In [1547]:
def classifier_fn(perturbed_texts):
    all_tokenized_sentences = []
    for text in perturbed_texts:
        temp_texts = []
        for t in tokenizer(text):
            temp_texts.append(voc[t])
        all_tokenized_sentences.append(temp_texts)
    
    max_len = max([len(tokens) for tokens in all_tokenized_sentences])
    for sentence in all_tokenized_sentences:
        while len(sentence) < max_len:
            sentence.append(0) # pad token
    all_tokenized_sentences = [torch.tensor(sentence) for sentence in all_tokenized_sentences]
    inputs = torch.stack(all_tokenized_sentences)
    
    model.eval()
    with torch.no_grad():
        predictions = model(inputs)
    return predictions

In [1548]:
def getExperimentID():
    date_info = datetime.datetime.now()
    testID    = '%d%02d%02d_%02d%02d' % (date_info.year, date_info.month, date_info.day, date_info.hour, date_info.minute)
    return testID

In [1549]:
def DefineModel(model_name, dim_per_layer=None, activation_per_layer=None, vocab_size=None, embed_dim=None, num_class=None):
    if 'text_ann' in model_name:
        # model = model_MLP_Text.Text_MLP(vocab_size, embed_dim, num_class)
        model = model_MLP_Text.Text_MLP(vocab_size)
    else:
        input_size = loader_train.dataset.get_number_of_features()
        if 'ann' in model_name:
            dim_per_layer = [input_size] + dim_per_layer
            model         = model_MLP.MLP(dim_per_layer, activation_per_layer)
        elif model_name == 'lr':
            dim_per_layer = [input_size] + dim_per_layer
            model         = LogisticRegression(dim_per_layer[0], dim_per_layer[1])
    
    return model

In [1555]:
files_dirs = [
    '20240612_202637_gpt-4-0125-preview_nshot16_k3_prompt-senti_classif_remove_words_v1_amazon_1000_text_ann',
    '20240613_014033_gpt-4-0125-preview_nshot16_k3_prompt-senti_classif_remove_words_v1_yelp_text_ann',
    '20240613_022348_gpt-4-0125-preview_nshot16_k3_prompt-senti_classif_remove_words_v1_imdb_text_ann',
    '20240613_143413_gpt-4_nshot16_k3_prompt-senti_classif_remove_words_v1_yelp_text_ann',
    '20240613_144431_gpt-4_nshot16_k3_prompt-senti_classif_remove_words_pgicl_yelp_text_ann',
    '20240613_151903_gpt-4_nshot16_k3_prompt-senti_classif_remove_words_v1_amazon_1000_text_ann',
    '20240613_152935_gpt-4_nshot16_k3_prompt-senti_classif_remove_words_pgicl_amazon_1000_text_ann',
    '20240613_160409_gpt-4_nshot16_k3_prompt-senti_classif_remove_words_v1_imdb_text_ann',
    '20240613_161500_gpt-4_nshot16_k3_prompt-senti_classif_remove_words_pgicl_imdb_text_ann',
    '20240613_232429_gpt-4-0125-preview_nshot16_k3_prompt-senti_classif_remove_words_pgicl_yelp_text_ann',
    '20240614_000552_gpt-4-0125-preview_nshot16_k3_prompt-senti_classif_remove_words_pgicl_amazon_1000_text_ann',
    '20240614_004547_gpt-4-0125-preview_nshot16_k3_prompt-senti_classif_remove_words_pgicl_imdb_text_ann'
 ]

for files_dir in files_dirs:
    files_dir = f'outputs/LLM_QueryAndReply/{files_dir}/'
    
    # list files in directory that end in summary.txt
    txt_file_replies = glob.glob(files_dir + '*summary.txt')
    n_shot = int(files_dir.split('_')[4].split('shot')[1])
    min_num_tokens_needed = int(np.log2(n_shot))
    # get the gpt model version
    llm_name = files_dir.split('/')[-2].split('_')[2]
    data_name = files_dir.split('/')[-2].split('_')[-3]
    prompt_strategy = files_dir.split('/')[-2].split('_')[-4]
    if data_name == '1000':
        data_name = 'amazon_1000'
        prompt_strategy = files_dir.split('/')[-2].split('_')[-5]
    
    print(n_shot, llm_name, data_name, prompt_strategy)
    
    # sort files so that they're read in numerical order: 0_gpt-4-0125-preview_ANN_L_beauty_summary.txt, 1_gpt-4-0125-preview_ANN_L_beauty_summary.txt, etc.
    txt_file_replies = sorted(txt_file_replies, key=lambda x: int(x.split('/')[-1].split('_')[0]))
    # txt_file_replies
    # read in each file
    replies = []
    for file in txt_file_replies:
        with open(file, 'r') as f:
            replies.append(f.read())
    # get the eval_idx from reply
    # '...eval_idx:\t\t1...'
    eval_inds = [int(reply.split('eval_idx:\t\t')[1].split('\n')[0]) for reply in replies]
    # strip and trailing \n get the last line of the file
    replies = [reply.strip().split('\n')[-1].split(',') for reply in replies]
    #strip each word of leading and trailing whitespace
    replies = [[[word.strip() for word in reply]] for reply in replies]
    # strip Answer: from '['Answer: the', 'of', 'was']'
    for r, reply in enumerate(replies):
        word = reply[0][0]
        if 'Answer:' in word:
            replies[r][0][0] = word.split('Answer: ')[1]
        if 'Final rank:' in word:
            replies[r][0][0] = word.split('Final rank: ')[1]
        if 'final rank:' in word:
            replies[r][0][0] = word.split('final rank: ')[1]
        if 'final ranking:' in word:
            replies[r][0][0] = word.split('final ranking: ')[1]
        if 'Final Rank:' in word:
            replies[r][0][0] = word.split('Final Rank: ')[1]
        if 'Final Answer:' in word:
            replies[r][0][0] = word.split('Final Answer: ')[1]
        if 'Final ranking:' in word:
            replies[r][0][0] = word.split('Final ranking: ')[1]
    # look at the bad replies and see if they can be fixed
    for r, reply in enumerate(replies):
        if len(reply[0]) > 3:
            replies[r][0] = reply[0][:3]
    # remove any extra quotes around any of the replies
    for r, reply in enumerate(replies):
        for w, word in enumerate(reply[0]):
            if word != '':
                if word[0] == '"' and word[-1] == '"':
                    replies[r][0][w] = word[1:-1]
                if word[0] == "'" and word[-1] == "'":
                    replies[r][0][w] = word[1:-1]
    # get indices where theres not 3 words
    # get indices where there's strings with more than 1 word in each of the 3 words
    bad_reply_indices = [i for i, reply in enumerate(replies) if len(reply[0]) != 3 or len(reply[0][0].split()) > 1 or len(reply[0][1].split()) > 1 or len(reply[0][2].split()) > 1  or reply[0][0] == '' or reply[0][1] == '' or reply[0][2] == '']
    # for bad_reply_index in bad_reply_indices:
        # print('bad_reply_indices', bad_reply_index)
        # print('replies[bad_reply_indices]', replies[bad_reply_index])
    manual_bad_reply = ['won/spoil', '**so']
    # manual_bad_reply_index = [i for i, reply in enumerate(replies) if len(reply[0]) >= 3 and reply[0][0] in manual_bad_reply or reply[0][1] in manual_bad_reply or reply[0][2] in manual_bad_reply]
    manual_bad_reply_index = []
    for i, reply in enumerate(replies):
        if len(reply[0]) >= 3 and (reply[0][0] in manual_bad_reply or reply[0][1] in manual_bad_reply or reply[0][2] in manual_bad_reply):
            # print('manual_bad_reply_index', i)
            # print('replies[manual_bad_reply_index]', replies[i])
            manual_bad_reply_index.append(i)
    
    # add to bad reply indices
    bad_reply_indices += manual_bad_reply_index
    # bad_reply_indices
    # replies
    # remove unecessary ** from either beginning or end of each word if it's there
    for r, reply in enumerate(replies):
        for w, word in enumerate(reply[0]):
            if word != '':
                if word[0] == '*' and word[1] == '*':
                    replies[r][0][w] = word[2:]
                if word[-1] == '*' and word[-2] == '*':
                    replies[r][0][w] = word[:-2]
    # replies
    all_exps = replies
    # data_name      = 'imdb'
    model_name     = 'text_ann'
    base_model_dir = './models/ClassWeighted/'
    batch_size     = 1
    num_test_samps = 100
    model_dir, model_file_name = get_model_names(model_name, data_name, base_model_dir)
    # Load dataset
    loader_train, loader_val, loader_test = return_loaders(data_name=data_name, batch_size=batch_size, download=False)
    
    # X_train, y_train = torch.FloatTensor(loader_train.dataset.data), torch.LongTensor(loader_train.dataset.targets.to_numpy())
    # X_val, y_val = torch.FloatTensor(loader_val.dataset.data), torch.LongTensor(loader_val.dataset.targets.to_numpy())
    # X_test, y_test = torch.FloatTensor(loader_test.dataset.data), torch.LongTensor(loader_test.dataset.targets.to_numpy())
    X_train = [data[0] for data in loader_train.dataset]
    y_train = np.array([data[1] for data in loader_train.dataset])
    X_val   = [data[0] for data in loader_val.dataset]
    y_val   = np.array([data[1] for data in loader_val.dataset])
    X_test  = [data[0] for data in loader_test.dataset]
    y_test  = np.array([data[1] for data in loader_test.dataset])
    
    # Load model
    tokenizer, voc = get_tokenizer_and_vocab(X_train, y_train)
    model = DefineModel(model_name, vocab_size=len(voc))
    
    model.load_state_dict(torch.load(model_dir + model_file_name))
    model.eval()
    # model
    X_test_lime = [data[0] for data in X_test]
    X_train_lime = [data[0] for data in X_train]
    
    # keep the eval_inds of the test set
    X_test_lime = [X_test_lime[i] for i in eval_inds]
    # skip sentences with the following text
    manual_skip_sentence = ['cinematography-if']
    manual_skip_sentence_index = []
    for i, sentence in enumerate(X_test_lime):
        if manual_skip_sentence[0] in sentence:
            # print('manual_skip_sentence_index', i)
            # print('X_test_lime[manual_skip_sentence_index]', X_test_lime[i])
            manual_skip_sentence_index.append(i)
    # manual_skip_sentence_index
    k = 3
    experiment_id = getExperimentID()
    PGI_AUC = calculateFaithfulnessAUC_text(X_test_lime, all_exps, model, 0, num_test_samps, k, do_pgu=False, do_random_baseline=False)
    PGU_AUC = calculateFaithfulnessAUC_text(X_test_lime, all_exps, model, 0, num_test_samps, k, do_pgu=True, do_random_baseline=False)
    print('PGI:' + str(round(np.mean(PGI_AUC), 3)) + '+/-' + str(round(np.std(PGI_AUC)/np.sqrt(len(PGI_AUC)), 3)))
    print('PGU:' + str(round(np.mean(PGU_AUC), 3)) + '+/-' + str(round(np.std(PGU_AUC)/np.sqrt(len(PGU_AUC)), 3)))
    
    # save out to file
    with open(f'outputs/TextFaithfulnessResults/{data_name}/faithfulness_{experiment_id}_{llm_name}_{data_name}_{model_name}_{prompt_strategy}_AUC-k{k}.txt', 'w') as f:
        f.write('PGI:' + str(round(np.mean(PGI_AUC), 3)) + '+/-' + str(round(np.std(PGI_AUC)/np.sqrt(len(PGI_AUC)), 3)) + '\n')
        f.write('PGU:' + str(round(np.mean(PGU_AUC), 3)) + '+/-' + str(round(np.std(PGU_AUC)/np.sqrt(len(PGU_AUC)), 3)) + '\n')
    # print(PGI_AUC)

16 gpt-4-0125-preview amazon_1000 v1
Vocabulary size: 1580
Vocabulary size: 1580
PGI:0.39+/-0.041
PGU:0.252+/-0.037
16 gpt-4-0125-preview yelp v1
Vocabulary size: 1721
Vocabulary size: 1721
PGI:0.3+/-0.038
PGU:0.231+/-0.033
16 gpt-4-0125-preview imdb v1
Vocabulary size: 2595
Vocabulary size: 2595
PGI:0.295+/-0.04
PGU:0.263+/-0.039
16 gpt-4 yelp v1
Vocabulary size: 1721
Vocabulary size: 1721
PGI:0.3+/-0.036
PGU:0.293+/-0.037
16 gpt-4 yelp pgicl
Vocabulary size: 1721
Vocabulary size: 1721
PGI:0.349+/-0.039
PGU:0.243+/-0.033
16 gpt-4 amazon_1000 v1
Vocabulary size: 1580
Vocabulary size: 1580
PGI:0.409+/-0.038
PGU:0.222+/-0.036
16 gpt-4 amazon_1000 pgicl
Vocabulary size: 1580
Vocabulary size: 1580
PGI:0.386+/-0.038
PGU:0.238+/-0.036
16 gpt-4 imdb v1
Vocabulary size: 2595
Vocabulary size: 2595
PGI:0.332+/-0.038
PGU:0.205+/-0.031
16 gpt-4 imdb pgicl
Vocabulary size: 2595
Vocabulary size: 2595
PGI:0.318+/-0.043
PGU:0.251+/-0.038
16 gpt-4-0125-preview yelp pgicl
Vocabulary size: 1721
Vocabular